# 02 – Preprocessing (Credit Card Customers / BankChurners)

Trong notebook này, em sẽ:
- Load lại dữ liệu thô **chỉ bằng NumPy**.
- Tách các cột numeric và categorical.
- Kiểm tra và xử lý các giá trị thiếu / không hợp lệ (`Unknown`, rỗng, ...).
- Chuẩn hoá dữ liệu:
  - Cài đặt các hàm Normalization (min-max, log, decimal scaling).
  - Cài đặt Standardization (z-score) và chọn z-score cho mô hình.
- Biến các biến categorical thành one-hot vectors bằng NumPy.
- Chia dữ liệu thành train/test (chỉ dùng **NumPy**).
- Xử lý mất cân bằng bằng undersampling.
- Lưu dữ liệu đã xử lý vào `data/processed/` để dùng cho modeling.


# 0. Khai báo thư viện

In [16]:
import sys
sys.path.append("..")

import numpy as np

import src.data_processing as dp

# 1. Đọc dữ liệu

In [17]:
csv_path = "../data/raw/BankChurners.csv"

header, raw = dp.load_data(csv_path)
feature_cols, X_raw, y = dp.split_features_target(header, raw)
cat_cols, num_cols, cat_idx, num_idx = dp.get_default_feature_groups(feature_cols)

print("Raw shape:", raw.shape)
print("X_raw shape:", X_raw.shape)
print("y shape:", y.shape)
print("cat_cols:", cat_cols)
print("num_cols:", num_cols)

Raw shape: (10127, 23)
X_raw shape: (10127, 19)
y shape: (10127,)
cat_cols: ['Gender', 'Education_Level', 'Marital_Status', 'Income_Category', 'Card_Category']
num_cols: ['Customer_Age', 'Dependent_count', 'Months_on_book', 'Total_Relationship_Count', 'Months_Inactive_12_mon', 'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal', 'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt', 'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio']


# 2. Tiền xử lý các giá trị

## 2.1 Tách X_cat, X_num & kiểm tra missing / Unknown

In [18]:
X_cat = X_raw[:, cat_idx]
X_num_str = X_raw[:, num_idx]

print("X_cat shape:", X_cat.shape)
print("X_num_str shape:", X_num_str.shape)

# Kiểm tra xem numeric có missing kiểu "", "NA", ... hay không
missing_tokens = np.array(["", "NA", "NaN", "nan", "NULL", "null"])
mask_missing_num = np.isin(X_num_str, missing_tokens)
print("Số ô numeric missing:", np.sum(mask_missing_num))

# Chuyển numeric sang float (dataset này thường không có missing numeric)
X_num = X_num_str.astype(float)

# Missing categorical: "" hoặc "Unknown"
mask_unknown_cat = (X_cat == "") | (X_cat == "Unknown")
print("Số ô categorical Unknown/rỗng:", np.sum(mask_unknown_cat))


X_cat shape: (10127, 5)
X_num_str shape: (10127, 14)
Số ô numeric missing: 0
Số ô categorical Unknown/rỗng: 3380


## 2.2 Xử lý Unknown bằng mode

In [19]:
X_cat_filled = dp.fill_unknowns_in_categorical(X_cat)
X_cat = X_cat_filled
print("Sau khi điền Unknown/rỗng:")
print("Số ô categorical Unknown/rỗng:", np.sum((X_cat == "") | (X_cat == "Unknown")))
print("Các giá trị categorical sau khi điền Unknown/rỗng:")
for i, col in enumerate(cat_cols):
    unique_values = np.unique(X_cat[:, i])
    print(f"  {col}: {unique_values}")



Sau khi điền Unknown/rỗng:
Số ô categorical Unknown/rỗng: 0
Các giá trị categorical sau khi điền Unknown/rỗng:
  Gender: ['F' 'M']
  Education_Level: ['College' 'Doctorate' 'Graduate' 'High School' 'Post-Graduate'
 'Uneducated']
  Marital_Status: ['Divorced' 'Married' 'Single']
  Income_Category: ['$120K +' '$40K - $60K' '$60K - $80K' '$80K - $120K' 'Less than $40K']
  Card_Category: ['Blue' 'Gold' 'Platinum' 'Silver']


## 2.3 Thêm các cột mới (Feature engineering)

In [20]:
# Thêm 3 feature mới 
X_num_ext, num_cols_ext = dp.add_engineered_features(X_num, num_cols)

print("X_num_ext shape:", X_num_ext.shape)
print("Số numeric + engineered:", len(num_cols_ext))
print("Tên cột numeric mở rộng:", num_cols_ext)


X_num_ext shape: (10127, 17)
Số numeric + engineered: 17
Tên cột numeric mở rộng: ['Customer_Age', 'Dependent_count', 'Months_on_book', 'Total_Relationship_Count', 'Months_Inactive_12_mon', 'Contacts_Count_12_mon', 'Credit_Limit', 'Total_Revolving_Bal', 'Avg_Open_To_Buy', 'Total_Amt_Chng_Q4_Q1', 'Total_Trans_Amt', 'Total_Trans_Ct', 'Total_Ct_Chng_Q4_Q1', 'Avg_Utilization_Ratio', 'Amt_per_Trans', 'Inactive_Ratio', 'Rel_per_Month']


# 2.4 Chuẩn hóa numeric (z-score)

In [21]:
X_num_std, num_mean, num_std = dp.standardize(X_num_ext)

print("X_num_std shape:", X_num_std.shape)
print("Mean gần 0 (cột 0):", X_num_std[:, 0].mean())
print("Std gần 1 (cột 0):", X_num_std[:, 0].std())


X_num_std shape: (10127, 17)
Mean gần 0 (cột 0): 1.2348723362671831e-16
Std gần 1 (cột 0): 0.9999999999999931


## 2.4 Kiểm tra missing value của các trường categorical

In [22]:
X_cat_oh, classes_per_cat = dp.one_hot_encode_all(X_cat, cat_cols)

print("X_cat_oh shape:", X_cat_oh.shape)
for name, classes in classes_per_cat.items():
    print(f"{name}: {len(classes)} classes -> {classes}")


X_cat_oh shape: (10127, 20)
Gender: 2 classes -> ['F' 'M']
Education_Level: 6 classes -> ['College' 'Doctorate' 'Graduate' 'High School' 'Post-Graduate'
 'Uneducated']
Marital_Status: 3 classes -> ['Divorced' 'Married' 'Single']
Income_Category: 5 classes -> ['$120K +' '$40K - $60K' '$60K - $80K' '$80K - $120K' 'Less than $40K']
Card_Category: 4 classes -> ['Blue' 'Gold' 'Platinum' 'Silver']


In [23]:
X_processed = np.concatenate([X_num_std, X_cat_oh], axis=1)

print("X_processed shape:", X_processed.shape)
print("Số feature numeric+eng:", X_num_std.shape[1])
print("Số chiều one-hot:", X_cat_oh.shape[1])
print("Tổng số feature:", X_processed.shape[1])


X_processed shape: (10127, 37)
Số feature numeric+eng: 17
Số chiều one-hot: 20
Tổng số feature: 37


In [24]:
X_train_full, X_test, y_train_full, y_test = dp.train_test_split_np(
    X_processed, y, test_size=0.2, random_state=42
)

print("X_train_full:", X_train_full.shape, "y_train_full:", y_train_full.shape)
print("X_test:", X_test.shape, "y_test:", y_test.shape)
print("Churn trong train_full:", np.sum(y_train_full == 1))
print("Churn trong test:", np.sum(y_test == 1))


X_train_full: (8102, 37) y_train_full: (8102,)
X_test: (2025, 37) y_test: (2025,)
Churn trong train_full: 1300
Churn trong test: 327


In [25]:
X_train, y_train = dp.undersample(X_train_full, y_train_full, ratio=1.0)

print("X_train (balanced):", X_train.shape, "y_train:", y_train.shape)
print("Churn train:", np.sum(y_train == 1))
print("Existing train:", np.sum(y_train == 0))


X_train (balanced): (2600, 37) y_train: (2600,)
Churn train: 1300
Existing train: 1300


In [26]:
np.save("../data/processed/X_train.npy", X_train)
np.save("../data/processed/y_train.npy", y_train)
np.save("../data/processed/X_test.npy",  X_test)
np.save("../data/processed/y_test.npy",  y_test)

# Lưu thêm bản train_full nếu muốn dùng sau
np.save("../data/processed/X_train_full.npy", X_train_full)
np.save("../data/processed/y_train_full.npy", y_train_full)

print("Đã lưu file .npy vào data/processed/")


Đã lưu file .npy vào data/processed/
